In [71]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
import numpy as np
from datetime import datetime
import time

import MetaTrader5 as mt5
import pygame

pygame.init()

sound_file = "message-ringtone-magic.mp3"  # Replace with the path to your sound file
pygame.mixer.music.load(sound_file)
symbol = "XAUUSD_i"

# Set filling mode


# Initialize the bound
mt5.initialize()

# Set the symbol
symbol = "XAUUSD_i"
# Set the point
username = 'mmashgh4'
password = 'Amin1380ali'
tv = TvDatafeed(username, password)

ERROR:tvDatafeed.main:error while signin


In [72]:
def get_data():
    # get data
    df_XUD = tv.get_hist(symbol='XAUUSD',exchange='OANDA',interval=Interval.in_5_minute,n_bars=300)
    #correct timing
    offset = pd.Timedelta(hours=1)
    df_XUD.index = df_XUD.index - offset
    return df_XUD

In [73]:
gold_mini_change_to_set_order = 0.3
gold_value_stop = 0.1

In [74]:
def get_macd_12(df_XUD):


    # Assuming your DataFrame is named df
    short_window = 12  # Short-term EMA
    long_window = 26   # Long-term EMA
    signal_window = 9  # Signal line EMA

    # Calculate Short-term EMA
    df_XUD['EMA_short_12'] = df_XUD['close'].ewm(span=short_window, adjust=False).mean()

    # Calculate Long-term EMA
    df_XUD['EMA_long_12'] = df_XUD['close'].ewm(span=long_window, adjust=False).mean()

    # Calculate MACD line
    df_XUD['MACD_12'] = df_XUD['EMA_short_12'] - df_XUD['EMA_long_12']

    # Calculate Signal line (Signal line is typically a 9-period EMA of MACD values)
    df_XUD['Signal_Line_12'] = df_XUD['MACD_12'].ewm(span=signal_window, adjust=False).mean()

    # Calculate MACD Histogram
    df_XUD['MACD_Histogram'] = df_XUD['MACD_12'] - df_XUD['Signal_Line_12']

    # Determine MACD trend direction (Above/Below Zero Line)
    df_XUD['MACD_Trend_12'] = df_XUD['MACD_12'].apply(lambda x: 'Above' if x > 0 else 'Below')

    # Determine Signal_Line trend direction (Above/Below Zero Line)
    df_XUD['Signal_Line_Trend_12'] = df_XUD['Signal_Line_12'].apply(lambda x: 'Above' if x > 0 else 'Below')


    # Determine Histogram color (Green if MACD > Signal Line, Red otherwise)
    df_XUD['Hist_color_V1'] = df_XUD.apply(lambda row: 'green' if row['MACD_12'] > row['Signal_Line_12'] else 'red', axis=1)


    # Function to determine color based on conditions
    def determine_color(row):
        if row['MACD_Histogram'] > 0 and abs(row['MACD_Histogram']) > abs(row['MACD_Histogram_shifted']):
            return 'green'  # Signal Line is below MACD and Histogram is increasing
        elif row['MACD_Histogram'] > 0 and abs(row['MACD_Histogram']) < abs(row['MACD_Histogram_shifted']):
            return 'light green'  # Signal Line is above MACD and Histogram is decreasing
        elif row['MACD_Histogram'] < 0 and abs(row['MACD_Histogram']) > abs(row['MACD_Histogram_shifted']):
            return 'red'  # Signal Line is above MACD and Histogram is decreasing
        else:
            return 'light red'  # Signal Line is above MACD and Histogram is not decreasing

    # Calculate the shift of the MACD_Histogram column
    df_XUD['MACD_Histogram_shifted'] = df_XUD['MACD_Histogram'].shift(1)

    # Apply the determine_color function to create the color column
    df_XUD['Hist_color_V2'] = df_XUD.apply(determine_color, axis=1)

    # Function to determine color based on conditions
    def determine_color_finisher(row):
        if (row['Hist_color_V2_shifted'] ==  'light green' or row['Hist_color_V2_shifted'] ==  'green') and row['Hist_color_V2'] == "light red":
            return 'red'  # fix first hist candle
        elif (row['Hist_color_V2_shifted'] ==  'light red' or row['Hist_color_V2_shifted'] ==  'red') and row['Hist_color_V2'] == "light green":
            return 'green'  # fix first hist candle
        else:
            return  row['Hist_color_V2'] # return defulat

    # Calculate the shift of the MACD_Histogram column
    df_XUD['Hist_color_V2_shifted'] = df_XUD['Hist_color_V2'].shift(1)

    # Advaned level: GOD bless me
    df_XUD['Hist_color_V3_12'] = df_XUD.apply(determine_color_finisher, axis=1)


    # Create a cross counter column
    counter = 0
    counter_list = []
    prev_color = None

    for color in df_XUD['Hist_color_V1']:
        if prev_color is None or color == prev_color:
            counter += 1
        else:
            counter = 1
        counter_list.append(counter)
        prev_color = color

    df_XUD['cross_counter'] = counter_list
    return df_XUD

In [75]:
def get_macd_27(df_XUD):


    # Assuming your DataFrame is named df
    short_window = 27  # Short-term EMA
    long_window = 26   # Long-term EMA
    signal_window = 9  # Signal line EMA

    # Calculate Short-term EMA
    df_XUD['EMA_short_27'] = df_XUD['close'].ewm(span=short_window, adjust=False).mean()

    # Calculate Long-term EMA
    df_XUD['EMA_long_27'] = df_XUD['close'].ewm(span=long_window, adjust=False).mean()

    # Calculate MACD line
    df_XUD['MACD_27'] = df_XUD['EMA_short_27'] - df_XUD['EMA_long_27']

    # Calculate Signal line (Signal line is typically a 9-period EMA of MACD values)
    df_XUD['Signal_Line_27'] = df_XUD['MACD_27'].ewm(span=signal_window, adjust=False).mean()

    # Calculate MACD Histogram
    df_XUD['MACD_Histogram'] = df_XUD['MACD_27'] - df_XUD['Signal_Line_27']

    # Determine MACD trend direction (Above/Below Zero Line)
    df_XUD['MACD_Trend_27'] = df_XUD['MACD_27'].apply(lambda x: 'Above' if x > 0 else 'Below')

    # Determine Signal_Line trend direction (Above/Below Zero Line)
    df_XUD['Signal_Line_Trend_27'] = df_XUD['Signal_Line_27'].apply(lambda x: 'Above' if x > 0 else 'Below')


    # Determine Histogram color (Green if MACD > Signal Line, Red otherwise)
    df_XUD['Hist_color_V1'] = df_XUD.apply(lambda row: 'green' if row['MACD_27'] > row['Signal_Line_27'] else 'red', axis=1)


    # Function to determine color based on conditions
    def determine_color(row):
        if row['MACD_Histogram'] > 0 and abs(row['MACD_Histogram']) > abs(row['MACD_Histogram_shifted']):
            return 'green'  # Signal Line is below MACD and Histogram is increasing
        elif row['MACD_Histogram'] > 0 and abs(row['MACD_Histogram']) < abs(row['MACD_Histogram_shifted']):
            return 'light green'  # Signal Line is above MACD and Histogram is decreasing
        elif row['MACD_Histogram'] < 0 and abs(row['MACD_Histogram']) > abs(row['MACD_Histogram_shifted']):
            return 'red'  # Signal Line is above MACD and Histogram is decreasing
        else:
            return 'light red'  # Signal Line is above MACD and Histogram is not decreasing

    # Calculate the shift of the MACD_Histogram column
    df_XUD['MACD_Histogram_shifted'] = df_XUD['MACD_Histogram'].shift(1)

    # Apply the determine_color function to create the color column
    df_XUD['Hist_color_V2'] = df_XUD.apply(determine_color, axis=1)

    # Function to determine color based on conditions
    def determine_color_finisher(row):
        if (row['Hist_color_V2_shifted'] ==  'light green' or row['Hist_color_V2_shifted'] ==  'green') and row['Hist_color_V2'] == "light red":
            return 'red'  # fix first hist candle
        elif (row['Hist_color_V2_shifted'] ==  'light red' or row['Hist_color_V2_shifted'] ==  'red') and row['Hist_color_V2'] == "light green":
            return 'green'  # fix first hist candle
        else:
            return  row['Hist_color_V2'] # return defulat

    # Calculate the shift of the MACD_Histogram column
    df_XUD['Hist_color_V2_shifted'] = df_XUD['Hist_color_V2'].shift(1)

    # Advaned level: GOD bless me
    df_XUD['Hist_color_V3_27'] = df_XUD.apply(determine_color_finisher, axis=1)


    # Create a cross counter column
    counter = 0
    counter_list = []
    prev_color = None

    for color in df_XUD['Hist_color_V1']:
        if prev_color is None or color == prev_color:
            counter += 1
        else:
            counter = 1
        counter_list.append(counter)
        prev_color = color

    df_XUD['cross_counter'] = counter_list
    return df_XUD

In [76]:
def analize_data(df_XUD):
    max_len = len(df_XUD)
    # Add SMA 20
    window_size = 20
    df_XUD['SMA 20'] = df_XUD['close'].rolling(window=window_size).mean()

    # Calculate the difference between current and previous day's closing price
    df_XUD['price_diff_CO'] = df_XUD['close'] -  df_XUD['open']

    # Add a new column to indicate green (1) or red (0)
    df_XUD['candle color'] = df_XUD['price_diff_CO'].apply(lambda x: 'green' if x > 0 else 'red')

    df_XUD = get_macd_12(df_XUD)

    def determine_candle_from_sma20(row):
        if (row['SMA 20'] > row['low']  and row['SMA 20'] < row['close']  and row['candle color'] == 'green'):
            return 'buy' 
        elif (row['SMA 20'] > row['close']  and row['SMA 20'] < row['high']  and row['candle color'] == 'red') :
            return 'sell'  
        else:
            return  '' # return defulat

    df_XUD['inside candle (rule 1)'] = df_XUD.apply(determine_candle_from_sma20, axis=1)

    def determine_distance_sma20(row):
        if (row['inside candle (rule 1)'] == 'buy'):
            return (row['high'] - row['SMA 20'] ) 
        elif (row['inside candle (rule 1)'] == 'sell') :
            return row['SMA 20'] - row['low']
        else:
            return  'no position' # return defulat

    df_XUD['distance SMA 20 from high or close'] = df_XUD.apply(determine_distance_sma20, axis=1)


    # from distance where sma 20 get broke and the candle closed , it shouldn't bigger than 4 other candles 
    df_XUD['candle_size'] = df_XUD['high'] - df_XUD['low']
    df_XUD['candle_size_shifted1'] = df_XUD['candle_size'].shift(1)
    df_XUD['candle_size_shifted2'] = df_XUD['candle_size'].shift(2)
    df_XUD['candle_size_shifted3'] = df_XUD['candle_size'].shift(3)
    df_XUD['candle_size_shifted4'] = df_XUD['candle_size'].shift(4)

    def determine_rule2(row):
        if row['distance SMA 20 from high or close'] == 'no position':
            return "False"
        elif (row['distance SMA 20 from high or close'] < row['candle_size_shifted4'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted2'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted3'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted1']):
            return "True"  
        else:
            return  'False'

    df_XUD['rule_2'] = df_XUD.apply(determine_rule2, axis=1)  

    def determine_rule3_12(row):
        if (row['MACD_Trend_12'] == 'Above'  and row['Signal_Line_Trend_12'] == 'Above') :
            return "buy"
        elif (row['MACD_Trend_12'] == 'Above'  and row['Signal_Line_Trend_12'] == 'Below'):
            return "Half_buy"  
        elif (row['MACD_Trend_12'] == 'Below'  and row['Signal_Line_Trend_12'] == 'Below'):
            return "sell"  
        else:
            return  'Half_sell'

    df_XUD['rule_3_12'] = df_XUD.apply(determine_rule3_12, axis=1) 

    #for long sabz kamrang and for short germez kamrang?
    def determine_rule4_5(row):
        if (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3_12'] == 'light red' and row['cross_counter'] >= 4  ) :
            return "buy"
        elif (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3_12'] == 'light green' ) :
            return "buy"
        elif (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3_12'] == 'green' and row['cross_counter'] <= 2 ):
            return "buy"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3_12'] == 'light green' and row['cross_counter'] >= 4 ) :
            return "sell"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3_12'] == 'light red' ) :
            return "sell"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3_12'] == 'red' and row['cross_counter'] <= 2 ):
            return "sell"

        else:
            return  'unknown'

    df_XUD['rule_4_5'] = df_XUD.apply(determine_rule4_5, axis=1) 

    df_XUD = get_macd_27(df_XUD)
    def determine_rule3_27(row):
        if (row['MACD_Trend_27'] == 'Above'  and row['Signal_Line_Trend_27'] == 'Above') :
            return "buy"
        elif (row['MACD_Trend_27'] == 'Above'  and row['Signal_Line_Trend_27'] == 'Below'):
            return "Half_buy"  
        elif (row['MACD_Trend_27'] == 'Below'  and row['Signal_Line_Trend_27'] == 'Below'):
            return "sell"  
        else:
            return  'Half_sell'

    df_XUD['rule_3_27'] = df_XUD.apply(determine_rule3_27, axis=1) 

    #for long sabz kamrang and for short germez kamrang?
    def determine_rule4_5_macd27(row):
        if (row['inside candle (rule 1)'] == 'buy' and ( row['Hist_color_V3_27'] == 'light red' or row['Hist_color_V3_27'] == 'red') and row['cross_counter'] <= 3) :
            return "buy"
        elif (row['inside candle (rule 1)'] == 'sell'  and ( row['Hist_color_V3_27'] == 'light green' or row['Hist_color_V3_27'] == 'green') and row['cross_counter']<= 3) :
            return "sell"
        else:
            return  'unknown'

    df_XUD['rule_4_5_macd27'] = df_XUD.apply(determine_rule4_5_macd27, axis=1) 

    df_XUD['SL_sell_shifted1'] = df_XUD['high'].shift(1)
    df_XUD['SL_sell_shifted2'] = df_XUD['high'].shift(2)
    df_XUD['SL_sell_shifted3'] = df_XUD['high'].shift(3)
    df_XUD['SL_sell_shifted4'] = df_XUD['high'].shift(4)

    df_XUD['SL_buy_shifted1'] = df_XUD['low'].shift(1)
    df_XUD['SL_buy_shifted2'] = df_XUD['low'].shift(2)
    df_XUD['SL_buy_shifted3'] = df_XUD['low'].shift(3)
    df_XUD['SL_buy_shifted4'] = df_XUD['low'].shift(4)

    def determine_SL_buy(row):
       return min(row['SL_buy_shifted1'],row['SL_buy_shifted2'],row['SL_buy_shifted3'],row['SL_buy_shifted4']) - 0.1

    def determine_SL_sell(row):
       return max(row['SL_sell_shifted1'],row['SL_sell_shifted2'],row['SL_sell_shifted3'],row['SL_sell_shifted4']) + 0.1

    df_XUD['SL_sell'] = df_XUD.apply(determine_SL_sell, axis=1) 
    df_XUD['SL_buy'] = df_XUD.apply(determine_SL_buy, axis=1) 


    def determine_entry_point_buy(row):
        if (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "FALSE" and (row["rule_3_12"] == "buy" or row["rule_3_12"] == "Half_buy") and row["rule_4_5"] == "buy" and (row["entry_buy"] - row["SL_buy"]) >= gold_mini_change_to_set_order ):
            return row["SMA 20"] +  abs((row["high"]-row["SMA 20"])/2)
        else :
            return row["high"] + gold_value_stop

    def determine_entry_point_sell(row):
        if (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "FALSE" and (row["rule_3_12"] == "sell" or row["rule_3_12"] == "Half_sell")and row["rule_4_5"] == "sell"  and (row["SL_sell"] -   row["entry_sell"]) >= gold_mini_change_to_set_order):
            return row["SMA 20"] - abs((row["low"]-row["SMA 20"])/2)
        else:
            return row["low"] - gold_value_stop



    df_XUD['entry_sell'] =  df_XUD.apply(determine_entry_point_sell, axis=1) 
    df_XUD['entry_buy'] =  df_XUD.apply(determine_entry_point_buy, axis=1) 


    def determine_TP_buy(row):
       return 1.5 * (row["entry_buy"] - row["SL_buy"]) + row["entry_buy"]

    def determine_TP_sell(row):
       return  row["entry_sell"] -  1.5 * (row["SL_sell"] -   row["entry_sell"])

    df_XUD['TP_sell'] = df_XUD.apply(determine_TP_sell, axis=1) 
    df_XUD['TP_buy'] = df_XUD.apply(determine_TP_buy, axis=1) 


    def determine_finall_desions(row):
        # MACD 12
        if (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_3_12"] == "buy" and row["rule_4_5"] == "buy" and (row["entry_buy"] - row["SL_buy"]) >= gold_mini_change_to_set_order ):
            return "buy_12"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_3_12"] == "sell" and row["rule_4_5"] == "sell"  and (row["SL_sell"] -   row["entry_sell"]) >= gold_mini_change_to_set_order):
            return "sell_12"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_3_12"] == "Half_sell" and row["rule_4_5"] == "sell" and (row["SL_sell"] -   row["entry_sell"]) >= gold_mini_change_to_set_order):
            return "sell_12"
        elif (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_3_12"] == "Half_buy" and row["rule_4_5"] == "buy" and (row["entry_buy"] - row["SL_buy"]) >= gold_mini_change_to_set_order ):
            return "buy_12"
        # elif (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and row["rule_3_12"] == "buy" and row["rule_4_5"] == "buy" and (row["entry_buy"] - row["SL_buy"]) >= gold_mini_change_to_set_order ):
        #     return "buy_with_big_candle"
        # elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and row["rule_3_12"] == "sell" and row["rule_4_5"] == "sell"  and (row["SL_sell"] -   row["entry_sell"]) >= gold_mini_change_to_set_order):
        #     return "sell_with_big_candle"
        # elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and row["rule_3_12"] == "Half_sell" and row["rule_4_5"] == "sell" and (row["SL_sell"] -   row["entry_sell"]) >= gold_mini_change_to_set_order):
        #     return "sell_with_big_candle"
        # elif (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and row["rule_3_12"] == "Half_buy" and row["rule_4_5"] == "buy" and (row["entry_buy"] - row["SL_buy"]) >= gold_mini_change_to_set_order ):
        #     return "buy_with_big_candle"
        # MACD 27
        # elif (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_3_27"] == "buy" and row["rule_4_5_macd27"] == "buy" and (row["entry_buy"] - row["SL_buy"]) >= gold_mini_change_to_set_order ):
        #     return "buy_27"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_3_27"] == "sell" and row["rule_4_5_macd27"] == "sell"  and (row["SL_sell"] -   row["entry_sell"]) >= gold_mini_change_to_set_order):
            return "sell_27"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_3_27"] == "Half_sell" and row["rule_4_5_macd27"] == "sell" and (row["SL_sell"] -   row["entry_sell"]) >= gold_mini_change_to_set_order):
            return "sell_27"
        elif (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_3_27"] == "Half_buy" and row["rule_4_5_macd27"] == "buy" and (row["entry_buy"] - row["SL_buy"]) >= gold_mini_change_to_set_order ):
            return "buy_27"
        # elif (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and row["rule_3_27"] == "buy" and row["rule_4_5_macd27"] == "buy" and (row["entry_buy"] - row["SL_buy"]) >= gold_mini_change_to_set_order ):
        #     return "buy_with_big_candle"
        # elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and row["rule_3_27"] == "sell" and row["rule_4_5_macd27"] == "sell"  and (row["SL_sell"] -   row["entry_sell"]) >= gold_mini_change_to_set_order):
        #     return "sell_with_big_candle"
        # elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and row["rule_3_27"] == "Half_sell" and row["rule_4_5_macd27"] == "sell" and (row["SL_sell"] -   row["entry_sell"]) >= gold_mini_change_to_set_order):
        #     return "sell_with_big_candle"
        # elif (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and row["rule_3_27"] == "Half_buy" and row["rule_4_5_macd27"] == "buy" and (row["entry_buy"] - row["SL_buy"]) >= gold_mini_change_to_set_order ):
        #     return "buy_with_big_candle"
        else :
            return "None"

    df_XUD['finall_desions'] = df_XUD.apply(determine_finall_desions, axis=1) 
    df_XUD['duplicated'] = None
    df_XUD['sma_20_breaked'] = None
    df_XUD['toched'] = None


    def cancle_other_order_buy(df_XUD , index):
        for i in range(1 ,4):
            if max_len == (index + i):
                return -1
            if df_XUD['inside candle (rule 1)'][index + i] == "sell":
                df_XUD['sma_20_breaked'][index + i] ="breaked_sell"
                return i
            else  :
                 df_XUD['duplicated'] [index + i] = "cancle_new_buy"
        return i

    def cancle_other_order_sell(df_XUD , index):

        for i in range(1 ,4):
            if max_len == (index + i):
                return  -1
            if df_XUD['inside candle (rule 1)'][index + i] == "buy":
                df_XUD['sma_20_breaked'][index + i] ="breaked_buy"
                return   i
            else  :
                 df_XUD['duplicated'] [index + i] = "cancle_new_sell"
        return  i

    leap_time = 0
    for i in range(len(df_XUD)):
        if leap_time > 0:
            leap_time = leap_time -1
            continue

        if df_XUD['finall_desions'][i] == "buy_12" or df_XUD['finall_desions'][i] == "buy_27":
            leap_time = cancle_other_order_buy(df_XUD,i)
        elif df_XUD['finall_desions'][i] == "sell_27" or df_XUD['finall_desions'][i] == "sell_12":
            leap_time = cancle_other_order_sell(df_XUD,i)

    def determin_toched_buy(df_XUD , index,buy_entry):
        for i in range(1 ,4):
            if max_len == (index + i):
                break
            if df_XUD['sma_20_breaked'][index + i] == "breaked_sell":
                break
            if df_XUD['high'][index + i] >= buy_entry:
                df_XUD['toched'][[index + i]] = "in_buy"
                break

    def determin_toched_sell(df_XUD , index,sell_entry):
        for i in range(1 ,4):
            if max_len == (index + i):
                break
            if df_XUD['sma_20_breaked'][index + i] == "breaked_buy":
                break
            if df_XUD['low'][index + i] <= sell_entry:
                df_XUD['toched'][[index + i]] = "in_sell"
                break

    for i in range(len(df_XUD)):
        if (df_XUD['finall_desions'][i] == "buy_12" or df_XUD['finall_desions'][i] == "buy_27") and df_XUD['duplicated'][i] != "cancle_new_buy":
            determin_toched_buy(df_XUD,i,df_XUD['entry_buy'][i])
        elif (df_XUD['finall_desions'][i] == "sell_27" or df_XUD['finall_desions'][i] == "sell_12") and df_XUD['duplicated'][i] != "cancle_new_sell":
            determin_toched_sell(df_XUD,i,df_XUD['entry_sell'][i])

    # Deleting multiple columns
    columns_to_delete = ["SL_buy_shifted4","SL_buy_shifted3","SL_buy_shifted2","SL_buy_shifted1","SL_sell_shifted4","SL_sell_shifted3","SL_sell_shifted2","SL_sell_shifted1","Hist_color_V1", "MACD_Histogram_shifted", "Hist_color_V2" , "Hist_color_V2_shifted","candle_size_shifted4","candle_size_shifted3","candle_size_shifted2","candle_size_shifted1"]
    df_XUD = df_XUD.drop(columns_to_delete, axis=1)

    # print(df_XUD)

    # Save the DataFrame to a CSV file
    df_XUD.to_csv('gold.csv', index=True)

    return df_XUD

In [77]:
analize_data(get_data())

,symbol,open,high,low,close,volume,SMA 20,price_diff_CO,candle color,EMA_short_12,...,SL_sell,SL_buy,entry_sell,entry_buy,TP_sell,TP_buy,finall_desions,duplicated,sma_20_breaked,toched
datetime,,,,,,,,,,,,,,,,,,,,,
2023-10-17 10:40:00,OANDA:XAUUSD,1919.925,1920.410,1919.670,1920.145,238.0,NaN,0.220,green,1920.145000,...,NaN,NaN,1919.570,1920.510,NaN,NaN,None,None,None,None
2023-10-17 10:45:00,OANDA:XAUUSD,1920.145,1920.180,1919.255,1919.840,289.0,NaN,-0.305,red,1920.098077,...,1920.510,1919.570,1919.155,1920.280,1917.1225,1921.3450,None,None,None,None
2023-10-17 10:50:00,OANDA:XAUUSD,1919.840,1920.120,1918.995,1919.475,246.0,NaN,-0.365,red,1920.002219,...,1920.510,1919.155,1918.895,1920.220,1916.4725,1921.8175,None,None,None,None
2023-10-17 10:55:00,OANDA:XAUUSD,1919.475,1919.940,1919.335,1919.815,327.0,NaN,0.340,green,1919.973416,...,1920.510,1918.895,1919.235,1920.040,1917.3225,1921.7575,None,None,None,None
2023-10-17 11:00:00,OANDA:XAUUSD,1919.815,1920.000,1919.180,1919.565,450.0,NaN,-0.250,red,1919.910583,...,1920.510,1918.895,1919.080,1920.100,1916.9350,1921.9075,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-18 12:15:00,OANDA:XAUUSD,1945.145,1945.380,1943.970,1944.130,574.0,1940.07775,-1.015,red,1941.867492,...,1946.285,1939.065,1943.870,1945.480,1940.2475,1955.1025,None,None,None,None
2023-10-18 12:20:00,OANDA:XAUUSD,1944.130,1944.970,1943.575,1944.930,523.0,1940.40925,0.800,green,1942.338647,...,1946.285,1940.515,1943.475,1945.070,1939.2600,1951.9025,None,None,None,None
2023-10-18 12:25:00,OANDA:XAUUSD,1944.930,1945.540,1944.365,1944.365,456.0,1940.75975,-0.565,red,1942.650394,...,1946.285,1943.475,1944.265,1945.640,1941.2350,1948.8875,None,None,None,None


In [78]:
# SELL LIMIT
def sell_signal(price_entry ,tp ,sl ):
    symbol = "XAUUSD_i"
    print("Sell" , "entry price :" , price_entry,"TP :" ,tp,"SL :", sl)

    filling_type = mt5.symbol_info(symbol).filling_mode

    request_sell_limit = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": 0.1,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": price_entry,
        "tp": tp,
        "sl": sl, 
        "type_filling": filling_type,
        "type_time": mt5.ORDER_TIME_GTC,
    }
    info_order = mt5.order_send(request_sell_limit)
    print(info_order)
    if info_order is None:
           print("Order send failed. Error:", mt5.last_error())
    else:
           print("Order placed successfully. Order info:", info_order)



    if info_order.retcode != mt5.TRADE_RETCODE_DONE or info_order is None:
        print(f"Order sending failed. Retcode: {info_order.retcode}, Comment: {info_order.comment}")
        request_sell_market = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": 0.1,  # You can adjust the volume as needed
            "type": mt5.ORDER_TYPE_SELL,
            "price":  mt5.symbol_info(symbol).bid,
            "tp": tp,
            "sl": sl, 
            "type_filling": mt5.ORDER_FILLING_IOC,
            "type_time": mt5.ORDER_TIME_GTC,
        }
        info_order = mt5.order_send(request_sell_market)
        print("sell" , "entry price :" ,  mt5.symbol_info_tick(symbol).bid,"TP :" ,tp,"SL :", sl)
    else:
        print("Sell" , "entry price :" , price_entry,"TP :" ,tp,"SL :", sl)
    return info_order
#             "price":  mt5.symbol_info(symbol).bid,

# BUY LIMIT
def buy_signal(price_entry ,tp ,sl ):
    symbol = "XAUUSD_i"
    print("Buy" , "entry price :" , price_entry,"TP :" ,tp,"SL :", sl)

    filling_type = mt5.symbol_info(symbol).filling_mode

    # Define the request for a buy stop order
    request_buy_stop = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": 0.1,  # Order volume
        "tp": tp,  # Take Profit price
        "sl": sl,  # Stop Loss price
        "type": mt5.ORDER_TYPE_BUY_STOP,  # Order type - Buy Stop
        "price": price_entry,  # Price at which you want to enter the market
        "deviation": 5,  # Price deviation
        "type_filling": filling_type,
        "type_time": mt5.ORDER_TIME_GTC  # Good Till Cancelled
    }
    info_order = mt5.order_send(request_buy_stop)
    print(info_order)
    if info_order is None:
           print("Order send failed. Error:", mt5.last_error())
    else:
           print("Order placed successfully. Order info:", info_order)
    if info_order.retcode != mt5.TRADE_RETCODE_DONE or info_order is None:
        print(f"Order sending failed. Retcode: {info_order.retcode}, Comment: {info_order.comment}")
        request_buy_market = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": 0.1,  # You can adjust the volume as needed
        "type": mt5.ORDER_TYPE_BUY,  # Use BUY type for buying
        "price": mt5.symbol_info_tick(symbol).ask,
        "tp": tp,
        "sl": sl, 
        "type_filling": filling_type,  # Use Immediate or Cancel filling type
        "type_time": mt5.ORDER_TIME_GTC
         }
        info_order = mt5.order_send(request_buy_market)
        print("Buy" , "entry price :" ,  mt5.symbol_info_tick(symbol).bid,"TP :" ,tp,"SL :", sl)

    else:
        print("Buy" , "entry price :" , price_entry,"TP :" ,tp,"SL :", sl)
    return info_order


In [79]:
def extract_signal(df_XUD):
    if (df_XUD["finall_desions"].iloc[-1] == "buy_12" or df_XUD["finall_desions"].iloc[-1] == "buy_27") and df_XUD['duplicated'][-1] != "cancle_new_buy":
        return 1 , 3
    elif (df_XUD['finall_desions'].iloc[-1] == "sell_27" or df_XUD['finall_desions'].iloc[-1] == "sell_12") and df_XUD['duplicated'].iloc[-1] != "cancle_new_sell" :
        return -1 ,3
    else :
        return 0 , -1

In [80]:
def change_candle(df_XUD):
    if df_XUD["inside candle (rule 1)"].iloc[-1] == "buy":
        if df_XUD["finall_desions"].iloc[-2] == "sell" or df_XUD["finall_desions"].iloc[-3] == "sell" or df_XUD["finall_desions"].iloc[-4] == "sell" :
            return True
        else:
            return False
    elif df_XUD["inside candle (rule 1)"].iloc[-1] == "sell":
        if df_XUD["finall_desions"].iloc[-2] == "buy" or df_XUD["finall_desions"].iloc[-3] == "buy" or df_XUD["finall_desions"].iloc[-4] == "buy" :
            return True
        else:
            return False
        
    return False

In [81]:
def Cancel_Order(info_order):
    # REMOVE PENDING ORDER
    print("Cancel order")
    request = {
         "action" : mt5.TRADE_ACTION_REMOVE ,
          "order" : info_order.order ,
    }
    mt5.order_send(request)


In [82]:
# get prepared date
# Specify the path to your CSV file
csv_file_path = 'test.csv'


def ready_data():
    # Read the CSV file into a DataFrame using pandas
    df = pd.read_csv(csv_file_path)
    return df


In [83]:
def check_positions(df_XUDD , info_order ,set_timer_v2):
    order_ticket = info_order.order
    positions = mt5.positions_get(ticket=order_ticket)
    if positions:
         position = positions[0]
         if position.type == mt5.POSITION_TYPE_BUY or position.type == mt5.POSITION_TYPE_SELL:
             print("Position is open.")
             return set_timer_v2 
         else:
             print("Position is closed.")
             return set_timer_v2
    else:
        if df_XUDD["sma_20_breaked"].iloc[-1] != "":
            Cancel_Order(info_order)
            print("No position found with the specified order ticket.")
            return -1
    return -1

def check_positions_to_cancel(df_XUDD , info_order):
    order_ticket = info_order.order
    positions = mt5.positions_get(ticket=order_ticket)
    if positions:
         position = positions[0]
         if position.type == mt5.POSITION_TYPE_BUY or position.type == mt5.POSITION_TYPE_SELL:
             print("Position is open. 2")
         else:
             print("Position is closed.2 ")
    else:
            Cancel_Order(info_order)
            print("No position found with the specified order ticket.2")
    
     

In [84]:
import warnings

# To suppress all warnings:
warnings.filterwarnings("ignore")

# Or, to suppress specific warnings by category:
# For example, to suppress the DeprecationWarning:
warnings.filterwarnings("ignore", category=DeprecationWarning)




def main(counter):
    set_timer_v2 = -1
    while(True):
      current_time = datetime.now()
      minutes =int(str(current_time.minute))%5
      minutes_live = int(str(current_time.minute))
      signal = 0
      if (minutes == 0) and  current_time.second >= 1 and current_time.second <= 3:#
         print("found noti" , current_time.minute  , current_time.second)
         counter = counter + 1 
         df_XUDD = analize_data(get_data())
         # df_XUDD = ready_data()
         if set_timer_v2 > 0 and info_order != None:
            set_timer_v2 = check_positions(df_XUDD ,info_order,set_timer_v2)
         if set_timer_v2 == 0 and info_order != None:
             check_positions_to_cancel(df_XUDD ,info_order)
         set_timer_v2 = set_timer_v2 - 1

        #  df_XUDD = ready_data()
        # Access the last element in the "finall_desions" column
         last_element = df_XUDD["finall_desions"].iloc[-1]
         if last_element != "None":
            print("s1",current_time.minute  ,  last_element)
            # Create the signals
            signal , set_timer_v2 =  extract_signal(df_XUDD)
            print(signal)
         if signal == 0:
            time.sleep(5)
        # #  if change_candle(df_XUDD) == False
        #           print("s3")
        #        else :
        #           print("s4")
        #           Cancel_Order(info_order)


      if signal != 0:
        symbol = "XAUUSD_i"
        lot = 1  
        mt5.initialize()
        filling_type = mt5.symbol_info(symbol).filling_mode
        point = mt5.symbol_info(symbol).point
        # Create the signals
        pygame.mixer.music.play()
        if signal == 1:
           info_order = buy_signal(  round(df_XUDD["entry_buy"].iloc[-1],2),  round(df_XUDD["TP_buy"].iloc[-1],2),  round(df_XUDD["SL_buy"].iloc[-1],2) )
           open_order_time = datetime.now()
           print("open buy position" , open_order_time.hour , open_order_time.minute)
        elif signal == -1:
            info_order = sell_signal(  round(df_XUDD["entry_sell"].iloc[-1],2), round(df_XUDD["TP_sell"].iloc[-1],2), round(df_XUDD["SL_sell"].iloc[-1],2) )
            open_order_time = datetime.now()
            print("open sell position" , open_order_time.hour , open_order_time.minute)
        time.sleep(5)




        # Run the algorithm
counter = 0
if __name__ == "__main__" :
    main(counter)

found noti 40 1
found noti 45 1
found noti 50 1
found noti 55 1
found noti 0 1
found noti 5 1
found noti 10 1
found noti 15 1
found noti 20 1
found noti 25 1
found noti 30 1
found noti 35 1
found noti 40 1
found noti 45 1
found noti 50 1
found noti 55 1
found noti 0 1
found noti 5 1
found noti 10 1
found noti 15 1
found noti 20 1
s1 20 sell_27
-1
Sell entry price : 1944.16 TP : 1938.98 SL : 1947.62
OrderSendResult(retcode=10015, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid price', request_id=0, retcode_external=0, request=TradeRequest(action=5, magic=0, order=0, symbol='XAUUSD_i', volume=0.1, price=1944.16, stoplimit=0.0, sl=1947.62, tp=1938.98, deviation=0, type=5, type_filling=3, type_time=0, expiration=0, comment='', position=0, position_by=0))
Order placed successfully. Order info: OrderSendResult(retcode=10015, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid price', request_id=0, retcode_external=0, request=TradeRequest(acti

TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'